In [2]:
from __future__ import print_function
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
np.random.seed(1337)  # for reproducibility
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Embedding
from keras.layers import LSTM, SimpleRNN, GRU
from keras.datasets import imdb
from keras.utils.np_utils import to_categorical
from sklearn.metrics import (precision_score, recall_score,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
from sklearn import metrics
from sklearn.preprocessing import Normalizer
import h5py
from keras import callbacks
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger

In [3]:
traindata = pd.read_csv('../dataset/traindata.csv', index_col = 0)
testdata = pd.read_csv('../dataset/traindata.csv', index_col = 0)

In [4]:
traindata['target'].replace(['normal.','buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.'], [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],inplace=True)

In [5]:
testdata['target'].replace(['normal.','buffer_overflow.', 'loadmodule.', 'perl.', 'neptune.',
       'smurf.', 'guess_passwd.', 'pod.', 'teardrop.', 'portsweep.',
       'ipsweep.', 'land.', 'ftp_write.', 'back.', 'imap.', 'satan.',
       'phf.', 'nmap.', 'multihop.', 'warezmaster.', 'warezclient.',
       'spy.', 'rootkit.'], [0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],inplace=True)

In [6]:
from sklearn.preprocessing import LabelEncoder


In [7]:
train = traindata.select_dtypes(exclude=[np.number])
print(train.columns)
for feature in train.columns:
    traindata[feature] = LabelEncoder().fit_transform(traindata[feature])

In [8]:
test = testdata.select_dtypes(exclude=[np.number])
print(test.columns)
for feature in test.columns:
    testdata[feature] = LabelEncoder().fit_transform(testdata[feature])

In [9]:
trainD = traindata.groupby('target', group_keys=False).apply(lambda x: x.sample(frac=0.05))
testD = testdata.groupby('target', group_keys=False).apply(lambda x: x.sample(frac=0.07))


In [10]:
X = trainD.iloc[:,0:41]
Y = trainD.iloc[:,41]
C = testD.iloc[:,41]
T = testD.iloc[:,0:41]

In [11]:
trainX = np.array(X)
testT = np.array(T)

trainX.astype(float)
testT.astype(float)

scaler = Normalizer().fit(trainX)
trainX = scaler.transform(trainX)

scaler = Normalizer().fit(testT)
testT = scaler.transform(testT)

y_train = np.array(Y)
y_test = np.array(C)


X_train = np.array(trainX)
X_test = np.array(testT)

In [12]:
batch_size = 64


In [ ]:
print("1 Layer DNN")

In [12]:
# 1. define the network
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

2023-04-16 13:32:59.713883: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn1layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn1layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn1layer/dnn1layer_model.hdf5")

Epoch 1/100
386/386 [==============================] - ETA: 0s - loss: 0.0270 - accuracy: 0.9889
Epoch 1: loss improved from inf to 0.02698, saving model to kddresults/dnn1layer/checkpoint-01.hdf5
386/386 [==============================] - 9s 19ms/step - loss: 0.0270 - accuracy: 0.9889 - val_loss: 0.0304 - val_accuracy: 0.9881
Epoch 2/100
381/386 [============================>.] - ETA: 0s - loss: 0.0241 - accuracy: 0.9891
Epoch 2: loss improved from 0.02698 to 0.02414, saving model to kddresults/dnn1layer/checkpoint-02.hdf5
386/386 [==============================] - 7s 17ms/step - loss: 0.0241 - accuracy: 0.9891 - val_loss: 0.0265 - val_accuracy: 0.9885
Epoch 3/100
382/386 [============================>.] - ETA: 0s - loss: 0.0222 - accuracy: 0.9908
Epoch 3: loss improved from 0.02414 to 0.02217, saving model to kddresults/dnn1layer/checkpoint-03.hdf5
386/386 [==============================] - 7s 18ms/step - loss: 0.0222 - accuracy: 0.9907 - val_loss: 0.0239 - val_accuracy: 0.9870
Epoch

386/386 [==============================] - 3s 8ms/step - loss: 0.0066 - accuracy: 0.9981 - val_loss: 0.0088 - val_accuracy: 0.9979
Epoch 53/100
375/386 [============================>.] - ETA: 0s - loss: 0.0066 - accuracy: 0.9984
Epoch 53: loss improved from 0.00663 to 0.00650, saving model to kddresults/dnn1layer/checkpoint-53.hdf5
386/386 [==============================] - 3s 8ms/step - loss: 0.0065 - accuracy: 0.9984 - val_loss: 0.0093 - val_accuracy: 0.9974
Epoch 54/100
382/386 [============================>.] - ETA: 0s - loss: 0.0065 - accuracy: 0.9981
Epoch 54: loss improved from 0.00650 to 0.00642, saving model to kddresults/dnn1layer/checkpoint-54.hdf5
386/386 [==============================] - 3s 7ms/step - loss: 0.0064 - accuracy: 0.9981 - val_loss: 0.0088 - val_accuracy: 0.9981
Epoch 55/100
374/386 [============================>.] - ETA: 0s - loss: 0.0061 - accuracy: 0.9982
Epoch 55: loss improved from 0.00642 to 0.00629, saving model to kddresults/dnn1layer/checkpoint-55.hdf

Epoch 80/100
380/386 [============================>.] - ETA: 0s - loss: 0.0052 - accuracy: 0.9984
Epoch 80: loss did not improve from 0.00502
386/386 [==============================] - 4s 11ms/step - loss: 0.0051 - accuracy: 0.9985 - val_loss: 0.0086 - val_accuracy: 0.9975
Epoch 81/100
386/386 [==============================] - ETA: 0s - loss: 0.0054 - accuracy: 0.9984
Epoch 81: loss did not improve from 0.00502
386/386 [==============================] - 5s 14ms/step - loss: 0.0054 - accuracy: 0.9984 - val_loss: 0.0090 - val_accuracy: 0.9977
Epoch 82/100
384/386 [============================>.] - ETA: 0s - loss: 0.0051 - accuracy: 0.9984
Epoch 82: loss did not improve from 0.00502
386/386 [==============================] - 9s 23ms/step - loss: 0.0051 - accuracy: 0.9984 - val_loss: 0.0095 - val_accuracy: 0.9979
Epoch 83/100
385/386 [============================>.] - ETA: 0s - loss: 0.0051 - accuracy: 0.9983
Epoch 83: loss did not improve from 0.00502
386/386 [===========================

In [15]:
print("2 Layer DNN")

2 Layer DNN


In [17]:
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn2layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn2layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn2layer/dnn2layer_model.hdf5")

Epoch 1/100
386/386 [==============================] - ETA: 0s - loss: 0.0427 - accuracy: 0.9841
Epoch 1: loss improved from inf to 0.04270, saving model to kddresults/dnn2layer/checkpoint-01.hdf5
386/386 [==============================] - 56s 54ms/step - loss: 0.0427 - accuracy: 0.9841 - val_loss: 0.0289 - val_accuracy: 0.9889
Epoch 2/100
385/386 [============================>.] - ETA: 0s - loss: 0.0219 - accuracy: 0.9914
Epoch 2: loss improved from 0.04270 to 0.02189, saving model to kddresults/dnn2layer/checkpoint-02.hdf5
386/386 [==============================] - 19s 50ms/step - loss: 0.0219 - accuracy: 0.9913 - val_loss: 0.0216 - val_accuracy: 0.9904
Epoch 3/100
386/386 [==============================] - ETA: 0s - loss: 0.0181 - accuracy: 0.9932
Epoch 3: loss improved from 0.02189 to 0.01809, saving model to kddresults/dnn2layer/checkpoint-03.hdf5
386/386 [==============================] - 18s 46ms/step - loss: 0.0181 - accuracy: 0.9932 - val_loss: 0.0190 - val_accuracy: 0.9955
Ep

386/386 [==============================] - 25s 64ms/step - loss: 0.0059 - accuracy: 0.9980 - val_loss: 0.0079 - val_accuracy: 0.9985
Epoch 27/100
385/386 [============================>.] - ETA: 0s - loss: 0.0057 - accuracy: 0.9983
Epoch 27: loss improved from 0.00589 to 0.00565, saving model to kddresults/dnn2layer/checkpoint-27.hdf5
386/386 [==============================] - 20s 53ms/step - loss: 0.0056 - accuracy: 0.9983 - val_loss: 0.0087 - val_accuracy: 0.9982
Epoch 28/100
386/386 [==============================] - ETA: 0s - loss: 0.0059 - accuracy: 0.9983
Epoch 28: loss did not improve from 0.00565
386/386 [==============================] - 19s 50ms/step - loss: 0.0059 - accuracy: 0.9983 - val_loss: 0.0091 - val_accuracy: 0.9979
Epoch 29/100
386/386 [==============================] - ETA: 0s - loss: 0.0055 - accuracy: 0.9985
Epoch 29: loss improved from 0.00565 to 0.00552, saving model to kddresults/dnn2layer/checkpoint-29.hdf5
386/386 [==============================] - 22s 56ms/s

Epoch 54/100
386/386 [==============================] - ETA: 0s - loss: 0.0043 - accuracy: 0.9986
Epoch 54: loss did not improve from 0.00427
386/386 [==============================] - 21s 54ms/step - loss: 0.0043 - accuracy: 0.9986 - val_loss: 0.0099 - val_accuracy: 0.9986
Epoch 55/100
385/386 [============================>.] - ETA: 0s - loss: 0.0043 - accuracy: 0.9985
Epoch 55: loss did not improve from 0.00427
386/386 [==============================] - 19s 50ms/step - loss: 0.0043 - accuracy: 0.9985 - val_loss: 0.0096 - val_accuracy: 0.9984
Epoch 56/100
385/386 [============================>.] - ETA: 0s - loss: 0.0047 - accuracy: 0.9986
Epoch 56: loss did not improve from 0.00427
386/386 [==============================] - 18s 46ms/step - loss: 0.0047 - accuracy: 0.9986 - val_loss: 0.0101 - val_accuracy: 0.9984
Epoch 57/100
386/386 [==============================] - ETA: 0s - loss: 0.0051 - accuracy: 0.9985
Epoch 57: loss did not improve from 0.00427
386/386 [========================

386/386 [==============================] - 19s 48ms/step - loss: 0.0030 - accuracy: 0.9988 - val_loss: 0.0128 - val_accuracy: 0.9983
Epoch 83/100
386/386 [==============================] - ETA: 0s - loss: 0.0032 - accuracy: 0.9986
Epoch 83: loss did not improve from 0.00304
386/386 [==============================] - 16s 42ms/step - loss: 0.0032 - accuracy: 0.9986 - val_loss: 0.0126 - val_accuracy: 0.9987
Epoch 84/100
386/386 [==============================] - ETA: 0s - loss: 0.0034 - accuracy: 0.9988
Epoch 84: loss did not improve from 0.00304
386/386 [==============================] - 20s 53ms/step - loss: 0.0034 - accuracy: 0.9988 - val_loss: 0.0123 - val_accuracy: 0.9986
Epoch 85/100
386/386 [==============================] - ETA: 0s - loss: 0.0029 - accuracy: 0.9990
Epoch 85: loss improved from 0.00304 to 0.00289, saving model to kddresults/dnn2layer/checkpoint-85.hdf5
386/386 [==============================] - 20s 53ms/step - loss: 0.0029 - accuracy: 0.9990 - val_loss: 0.0116 - va

In [18]:
print("3 Layer DNN")

3 Layer DNN


In [ ]:
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn3layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn3layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=50, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn3layer/dnn3layer_model.hdf5")

In [14]:
print("4 Layer DNN")

4 Layer DNN


In [15]:
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn4layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn4layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=batch_size, epochs=50, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn4layer/dnn4layer_model.hdf5")

Epoch 1/50
386/386 [==============================] - ETA: 0s - loss: 0.0419 - accuracy: 0.9840
Epoch 1: loss improved from inf to 0.04193, saving model to kddresults/dnn4layer/checkpoint-01.hdf5
386/386 [==============================] - 16s 37ms/step - loss: 0.0419 - accuracy: 0.9840 - val_loss: 0.0267 - val_accuracy: 0.9887
Epoch 2/50
386/386 [==============================] - ETA: 0s - loss: 0.0232 - accuracy: 0.9917
Epoch 2: loss improved from 0.04193 to 0.02319, saving model to kddresults/dnn4layer/checkpoint-02.hdf5
386/386 [==============================] - 15s 39ms/step - loss: 0.0232 - accuracy: 0.9917 - val_loss: 0.0236 - val_accuracy: 0.9900
Epoch 3/50
385/386 [============================>.] - ETA: 0s - loss: 0.0192 - accuracy: 0.9927
Epoch 3: loss improved from 0.02319 to 0.01914, saving model to kddresults/dnn4layer/checkpoint-03.hdf5
386/386 [==============================] - 15s 39ms/step - loss: 0.0191 - accuracy: 0.9927 - val_loss: 0.0207 - val_accuracy: 0.9902
Epoch

384/386 [============================>.] - ETA: 0s - loss: 0.0073 - accuracy: 0.9980
Epoch 27: loss did not improve from 0.00654
386/386 [==============================] - 15s 39ms/step - loss: 0.0075 - accuracy: 0.9980 - val_loss: 0.0109 - val_accuracy: 0.9977
Epoch 28/50
386/386 [==============================] - ETA: 0s - loss: 0.0070 - accuracy: 0.9977
Epoch 28: loss did not improve from 0.00654
386/386 [==============================] - 14s 35ms/step - loss: 0.0070 - accuracy: 0.9977 - val_loss: 0.0107 - val_accuracy: 0.9977
Epoch 29/50
386/386 [==============================] - ETA: 0s - loss: 0.0061 - accuracy: 0.9982
Epoch 29: loss improved from 0.00654 to 0.00605, saving model to kddresults/dnn4layer/checkpoint-29.hdf5
386/386 [==============================] - 17s 43ms/step - loss: 0.0061 - accuracy: 0.9982 - val_loss: 0.0127 - val_accuracy: 0.9977
Epoch 30/50
385/386 [============================>.] - ETA: 0s - loss: 0.0064 - accuracy: 0.9980
Epoch 30: loss did not improve f

In [16]:
print("5 Layer DNN")

5 Layer DNN


In [17]:
model = Sequential()
model.add(Dense(1024,input_dim=41,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(768,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(512,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(256,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(128,activation='relu'))  
model.add(Dropout(0.01))
model.add(Dense(1))
model.add(Activation('sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
checkpointer = callbacks.ModelCheckpoint(filepath="kddresults/dnn5layer/checkpoint-{epoch:02d}.hdf5", verbose=1, save_best_only=True, monitor='loss')
csv_logger = CSVLogger('kddresults/dnn5layer/training_set_dnnanalysis.csv',separator=',', append=False)
model.fit(X_train, y_train, validation_data=(X_test, y_test),batch_size=batch_size, epochs=100, callbacks=[checkpointer,csv_logger])
model.save("kddresults/dnn5layer/dnn5layer_model.hdf5")

Epoch 1/100
386/386 [==============================] - ETA: 0s - loss: 0.0404 - accuracy: 0.9859
Epoch 1: loss improved from inf to 0.04043, saving model to kddresults/dnn5layer/checkpoint-01.hdf5
386/386 [==============================] - 18s 42ms/step - loss: 0.0404 - accuracy: 0.9859 - val_loss: 0.0270 - val_accuracy: 0.9890
Epoch 2/100
386/386 [==============================] - ETA: 0s - loss: 0.0250 - accuracy: 0.9902
Epoch 2: loss improved from 0.04043 to 0.02498, saving model to kddresults/dnn5layer/checkpoint-02.hdf5
386/386 [==============================] - 17s 44ms/step - loss: 0.0250 - accuracy: 0.9902 - val_loss: 0.0258 - val_accuracy: 0.9889
Epoch 3/100
385/386 [============================>.] - ETA: 0s - loss: 0.0186 - accuracy: 0.9915
Epoch 3: loss improved from 0.02498 to 0.01858, saving model to kddresults/dnn5layer/checkpoint-03.hdf5
386/386 [==============================] - 16s 41ms/step - loss: 0.0186 - accuracy: 0.9915 - val_loss: 0.0220 - val_accuracy: 0.9923
Ep

385/386 [============================>.] - ETA: 0s - loss: 0.0081 - accuracy: 0.9981
Epoch 28: loss did not improve from 0.00673
386/386 [==============================] - 18s 46ms/step - loss: 0.0081 - accuracy: 0.9981 - val_loss: 0.0112 - val_accuracy: 0.9976
Epoch 29/100
385/386 [============================>.] - ETA: 0s - loss: 0.0072 - accuracy: 0.9982
Epoch 29: loss did not improve from 0.00673
386/386 [==============================] - 21s 56ms/step - loss: 0.0072 - accuracy: 0.9982 - val_loss: 0.0085 - val_accuracy: 0.9978
Epoch 30/100
386/386 [==============================] - ETA: 0s - loss: 0.0070 - accuracy: 0.9979
Epoch 30: loss did not improve from 0.00673
386/386 [==============================] - 23s 59ms/step - loss: 0.0070 - accuracy: 0.9979 - val_loss: 0.0110 - val_accuracy: 0.9971
Epoch 31/100
385/386 [============================>.] - ETA: 0s - loss: 0.0066 - accuracy: 0.9981
Epoch 31: loss improved from 0.00673 to 0.00660, saving model to kddresults/dnn5layer/chec

385/386 [============================>.] - ETA: 0s - loss: 0.0052 - accuracy: 0.9984
Epoch 56: loss did not improve from 0.00441
386/386 [==============================] - 16s 41ms/step - loss: 0.0052 - accuracy: 0.9984 - val_loss: 0.0116 - val_accuracy: 0.9985
Epoch 57/100
385/386 [============================>.] - ETA: 0s - loss: 0.0047 - accuracy: 0.9984
Epoch 57: loss did not improve from 0.00441
386/386 [==============================] - 16s 43ms/step - loss: 0.0047 - accuracy: 0.9984 - val_loss: 0.0115 - val_accuracy: 0.9983
Epoch 58/100
386/386 [==============================] - ETA: 0s - loss: 0.0052 - accuracy: 0.9985
Epoch 58: loss did not improve from 0.00441
386/386 [==============================] - 18s 47ms/step - loss: 0.0052 - accuracy: 0.9985 - val_loss: 0.0109 - val_accuracy: 0.9979
Epoch 59/100
385/386 [============================>.] - ETA: 0s - loss: 0.0051 - accuracy: 0.9983
Epoch 59: loss did not improve from 0.00441
386/386 [==============================] - 22s

386/386 [==============================] - 33s 86ms/step - loss: 0.0048 - accuracy: 0.9985 - val_loss: 0.0123 - val_accuracy: 0.9981
Epoch 85/100
386/386 [==============================] - ETA: 0s - loss: 0.0036 - accuracy: 0.9989
Epoch 85: loss did not improve from 0.00331
386/386 [==============================] - 30s 78ms/step - loss: 0.0036 - accuracy: 0.9989 - val_loss: 0.0138 - val_accuracy: 0.9984
Epoch 86/100
385/386 [============================>.] - ETA: 0s - loss: 0.0068 - accuracy: 0.9984
Epoch 86: loss did not improve from 0.00331
386/386 [==============================] - 29s 75ms/step - loss: 0.0067 - accuracy: 0.9984 - val_loss: 0.0126 - val_accuracy: 0.9984
Epoch 87/100
386/386 [==============================] - ETA: 0s - loss: 0.0063 - accuracy: 0.9985
Epoch 87: loss did not improve from 0.00331
386/386 [==============================] - 35s 92ms/step - loss: 0.0063 - accuracy: 0.9985 - val_loss: 0.0171 - val_accuracy: 0.9986
Epoch 88/100
386/386 [====================